# Leave-one-site-out GBM prediction accuracy

In [4]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import pickle

In [5]:
data_1 = pd.read_csv('../Data/CO/13-N1/outputs/yield_stability/13-N1_yield_stability_indexes_with_terrain.csv')
data_1.columns = [col.replace("_CO", "") for col in data_1.columns]
data_1['Site'] = 'CO'
data_1['Field'] = '13-N1'
print(data_1.shape)
data_2 = pd.read_csv('../Data/CO/13-N2/outputs/yield_stability/13-N2_yield_stability_indexes_with_terrain.csv')
data_2.columns = [col.replace("_CO", "") for col in data_2.columns]
data_2['Site'] = 'CO'
data_2['Field'] = '13-N2'
print(data_2.shape)
data_3 = pd.read_csv('../Data/MD/Hevelow5/outputs/yield_stability/Hevelow5_yield_stability_indexes_with_terrain.csv')
data_3.columns = [col.replace("_MD", "") for col in data_3.columns]
data_3['Site'] = 'MD'
data_3['Field'] = 'Hevelow5'
print(data_3.shape)
data_4 = pd.read_csv('../Data/MD/Kennedyville2/outputs/yield_stability/Kennedyville2_yield_stability_indexes_with_terrain.csv')
data_4.columns = [col.replace("_MD", "") for col in data_4.columns]
data_4['Site'] = 'MD'
data_4['Field'] = 'Kennedyville2'
print(data_4.shape)
data_5 = pd.read_csv('../Data/TX/6-12/outputs/yield_stability/6-12_yield_stability_indexes_with_terrain.csv')
data_5.columns = [col.replace("_TX", "") for col in data_5.columns]
data_5['Site'] = 'TX'
data_5['Field'] = '6-12'
print(data_5.shape)
data_6 = pd.read_csv('../Data/TX/Y10/outputs/yield_stability/Y10_yield_stability_indexes_with_terrain.csv')
data_6.columns = [col.replace("_TX", "") for col in data_6.columns]
data_6['Site'] = 'TX'
data_6['Field'] = 'Y10'
print(data_6.shape)

dfs = [data_1, data_2, data_3, data_4, data_5, data_6]

common_columns = set(dfs[0].columns)
for df in dfs[1:]:
    common_columns.intersection_update(df.columns)

common_columns = sorted(list(common_columns))
dfs_aligned = [df.reindex(columns = common_columns) for df in dfs]

data = pd.concat(dfs_aligned, ignore_index = True)
print(data.shape)
data = data[data['CV'] != 0]
data = data[data['Mean'] >= 0]
print(set(data['geomorphons_LP10']))
data = pd.get_dummies(data, columns=['geomorphons_LP10'])

# Converting some to SD
data['SD'] = data['CV'] * data['Mean']
data['One_plus_Ymean_minus_SD_over_Ymean_plus_SD_over_2'] = (1 + (data['Mean'] - data['SD'])/(data['Mean'] + data['SD']))/2
data['Ymean_times_One_plus_Ymean_minus_SD_over_Ymean_plus_SD_over_2'] = data['Mean'] * ((1 + (data['Mean'] - data['SD'])/(data['Mean'] + data['SD']))/2)
data.to_csv('full_dataset.csv')
print(data.shape)

(33278, 144)
(34059, 144)
(9235, 143)
(10829, 143)
(2844, 144)
(3056, 144)
(93301, 143)
{1, 2, 3, 4, 5, 6, 7, 8, 9}
(92488, 154)


In [6]:
# Define the target variable and terrain attributes
target_variable = 'Ymean_times_One_plus_Ymean_minus_CV_over_Ymean_plus_CV_over_2'

terrain_attributes = [
    'Aspect', 'Aspect_LP10', 'Aspect_LP30', 'Aspect_ss_5',
       'DEM5', 'DEM5_ns', 'DEM_ss_5', 'DEM_ss_5_ns', 'FW', 'FW_gs_LP10',
       'FW_ss_5', 'FlowAccD8', 'FlowAccD8_LP10', 'FlowAccMFD1.1',
       'FlowAccMFD1.1_LP10', 'FlowAccMFD1.1_ss_5', 'LP10', 'LP10_ns',
       'LP30', 'MRRTF_LP10', 'MRRTF_ss_5', 'MRVBF_LP10', 'MRVBF_ss_5',
       'Midslope_LP10', 'NegOpen', 'NegOpen_LP10', 'NegOpen_LP30',
       'NegOpen_ss_5', 'NormalizedHeight_LP10', 'PosOpen',
       'PosOpen_LP10', 'PosOpen_LP30', 'PosOpen_ss_5',
       'ProfileCurvature_LP30', 'SCA_D8_LP10', 'SCA_MFD1.1',
       'SCA_MFD1.1_LP10', 'SCA_MFD1.1_ss_5', 'SWI10', 'SWI1E16', 'SWI1E2',
       'SWI1E2_LP10', 'SWI1E2_LP30', 'SWI1E4', 'SWI1E8', 'Slope',
       'SlopeHeight_LP10', 'Slope_LP10', 'Slope_LP30', 'Slope_ss_5',
       'StandardizedHeight_LP10', 'TPI_LP10', 'TPI_LP30', 'TPI_ss_5',
       'TWI_D8_LP10', 'TWI_MFD1.1', 'TWI_MFD1.1_LP10', 'TWI_MFD1.1_ss_5',
       'TangentCurvature_LP30', 'ValleyDepth_LP10', 'channel_network', 'channel_route', 'geomorphons_LP30',
       'hillshade', 'vdist_chn_network', 'wind_shelter_LP10',
       'wind_shelter_ss_5', 'geomorphons_LP10_1', 'geomorphons_LP10_3',
       'geomorphons_LP10_4', 'geomorphons_LP10_5', 'geomorphons_LP10_6',
       'geomorphons_LP10_7', 'geomorphons_LP10_8', 'geomorphons_LP10_9',
    'SoilBulkDensity_0-5cm', 'SoilBulkDensity_100-200cm',
       'SoilBulkDensity_15-30cm', 'SoilBulkDensity_30-60cm',
       'SoilBulkDensity_5-15cm', 'SoilBulkDensity_60-100cm',
       'SoilClay_0-5cm', 'SoilClay_100-200cm', 'SoilClay_15-30cm',
       'SoilClay_30-60cm', 'SoilClay_5-15cm', 'SoilClay_60-100cm',
       'SoilSilt_0-5cm', 'SoilSilt_100-200cm', 'SoilSilt_15-30cm',
       'SoilSilt_30-60cm', 'SoilSilt_5-15cm', 'SoilSilt_60-100cm',
       'Soil_ksat_0-5cm', 'Soil_ksat_100-200cm', 'Soil_ksat_15-30cm',
       'Soil_ksat_30-60cm', 'Soil_ksat_5-15cm', 'Soil_ksat_60-100cm',
       'Soil_om_0-5cm', 'Soil_om_100-200cm', 'Soil_om_15-30cm',
       'Soil_om_30-60cm', 'Soil_om_5-15cm', 'Soil_om_60-100cm',
       'SoilpH_0-5cm', 'SoilpH_100-200cm', 'SoilpH_15-30cm',
       'SoilpH_30-60cm', 'SoilpH_5-15cm', 'SoilpH_60-100cm',
       'Soilsand_0-5cm', 'Soilsand_100-200cm', 'Soilsand_15-30cm',
       'Soilsand_30-60cm', 'Soilsand_5-15cm', 'Soilsand_60-100cm',
    'AnnualPrecipitation', 'PrecipColdestQuarter', 'PrecipDriestMonth', 'PrecipDriestQuarter',
       'PrecipSeasonality', 'PrecipWarmestQuarter', 'PrecipWettestMonth',
       'PrecipWettestQuarter', 'MaxTempWarmMonth','MeanTempColdQuarter', 'MeanTempDryQuarter', 'MeanTempWarmQuarter',
       'MeanTempWetQuarter', 'MinTempColdMonth', 'TempAnnualRange', 'TemperatureSeasonality', 'isothermality',
       'meanDiurnalRange', 'annualMeanTemperature', 'NDVI_avg'
]

In [7]:
# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Initialize LightGBM Regressor
lgb_regressor = lgb.LGBMRegressor(random_state=42)

# Store results
results = {}

# List of fields
fields = data['Field'].unique()

# GroupKFold cross-validation
gkf = GroupKFold(n_splits=len(fields))

X = data[terrain_attributes]
y = data[target_variable]
groups = data['Field']

# Impute missing values
X = imputer.fit_transform(X)

# Initialize Standard Scaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 50, 100],
    'min_child_samples': [20, 30, 40],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Perform group cross-validation with hyperparameter tuning
for train_index, test_index in gkf.split(X, y, groups):
    field = data.iloc[test_index]['Field'].unique()[0]
    print(f'Processing field {field}...')

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Perform hyperparameter tuning using GridSearchCV
    grid_search = GridSearchCV(estimator=lgb_regressor, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Get the best model from grid search
    best_lgb = grid_search.best_estimator_

    # Save the model
    with open(f'models/model_{target_variable}_field_{field}.pkl', 'wb') as model_file:
        pickle.dump(best_lgb, model_file)
    
    pd.DataFrame(X_train).to_csv(f'models/X_train_{target_variable}_field_{field}.csv', index=False)

    # Cross-validation scores
    mean_cv_score = grid_search.best_score_

    # Predict on training and test sets
    y_train_pred = best_lgb.predict(X_train)
    y_test_pred = best_lgb.predict(X_test)
    
    # Calculate R² and RMSE for training and test sets
    train_r2 = r2_score(y_train, y_train_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    # Store feature importances and scores
    results[field] = {
        'feature_importances': best_lgb.feature_importances_,
        'cv_r2': mean_cv_score,
        'train_r2': train_r2,
        'train_rmse': train_rmse,
        'test_r2': test_r2,
        'test_rmse': test_rmse
    }

    print(f'Model for field {field}:')
    print(f'Cross-Validation R²: {mean_cv_score}')
    print(f'Training R²: {train_r2}')
    print(f'Training RMSE: {train_rmse}')
    print(f'Test R²: {test_r2}')
    print(f'Test RMSE: {test_rmse}')
    print('Feature Importances:', best_lgb.feature_importances_)
    print()

# Convert results to DataFrames for easier visualization
importance_df = pd.DataFrame({
    field: results[field]['feature_importances'] for field in fields
}, index=terrain_attributes)

scores_df = pd.DataFrame({
    'CV_R2': {field: results[field]['cv_r2'] for field in fields},
    'Train_R2': {field: results[field]['train_r2'] for field in fields},
    'Train_RMSE': {field: results[field]['train_rmse'] for field in fields},
    'Test_R2': {field: results[field]['test_r2'] for field in fields},
    'Test_RMSE': {field: results[field]['test_rmse'] for field in fields}
})

Processing field 13-N2...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 5.550567 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 27455
[LightGBM] [Info] Number of data points in the train set: 46986, number of used features: 137
[LightGBM] [Info] Start training from score 0.281946
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

Exception ignored on calling ctypes callback function: <function _log_callback at 0x7f850193b380>
Traceback (most recent call last):
  File "/home/bostevens/anaconda3/envs/py311/lib/python3.11/site-packages/lightgbm/basic.py", line 257, in _log_callback
    def _log_callback(msg: bytes) -> None:
    
KeyboardInterrupt: 


KeyboardInterrupt: 